## DSCI100 GROUP PROJECT

##### Group Members: Aasha (**add student#**), Jamie (70834411), Nathelie (**add student#**), April (**add student#**)
##### Group Number: 007-39

In [1]:
#loading packages
library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

## Introduction
#### Background Information:
This project analyzes real-world data collected from a Minecraft research server (plaicraft.ai), run by the PLAI group of the UBC Computer Science department.             
The goal is to help researchers find out which "kinds" of players are most likely to contribute a large amount of data, so that they can maximize the amount of data collected with their limited resources.              
Specifically, the project will focus on predicting how much playing time a player contributes to the research based on their age and experience. The longer the play time, the more data the participant contributes. 

#### Research Question: Can player age and player experience predict the duration of playing?

#### Dataset Description
Two datasets were provided for this project, namely "players.csv" (assigned to `players`), and "sessions.csv" (assigned to `sessions`).           
`players` stores information about each research participant, while `sessions` stores information about each unique playing session.         

In [ ]:
#initial data loading
players <- read_csv("https://raw.githubusercontent.com/jamiekyh/dsci100_project/refs/heads/main/players.csv")
head(players)

sessions <- read_csv("https://raw.githubusercontent.com/jamiekyh/dsci100_project/refs/heads/main/sessions.csv")
head(sessions)

#### Dataset Description: `players`
Now that we have loaded the entire comma-separated values file, we need to narrow the data down to what we need to answer the question.        
To answer our question, we will be using the `players` dataset to obtain data on players' age, experience and hours played.             

##### Summary
Let's take the summary and break down what is in the dataset:


In [ ]:
summary(players)

To answer our question we will be using the dataset `players` to gain data on each players age, experience and hours played. Lets take the summary and breakdown whats in the dataset:

Tabulated summary of `players`:
| Variable         | Type   | Description |number of rows|
|-----------------|--------|-------------|--------|
| experience      | character | Player's self-reported experience level (e.g., Beginner, Regular, Veteran) | 196 |
| subscribe | logical | Whether the player is subscribed to a game-related newsletter | 196 |
| hashedEmail       | character | Unique identifier for each player | 196 |
| played_hours    | double | Total duration the player spent in the game | 196 |
| name    | character | Name of player | 196 |
| gender          | character | Player's gender (e.g., Male, Female, Other) | 196 |
| Age            | double | Player’s age in years| 196 |

The table above gives us a clear and legible summary of the dataset.                     
Each variable is named, along with the data type, description of the variable, and the number of entries for each variable.            

There are a few things to note from this table:    

                   
First, `Age` is of double type, although the ages listed are integers. Although the data type is not a problem for our data analysis, there is a hidden assumption that age is a discrete numerical variable, and thus should be treated as a categorical variable rather than a continuous one.                       
To address this problem, the distance between two ages will be defined as the absolute value of the difference between the two age values.
                 
                
Secondly, both of our predictors (`experience` and `Age`) are technically categorical. To be able to use these variables in a KNN regression model, these categorical data have to be converted into numerical values, with levels assigned according to increasing player experience and increasing player age.
While this "levelling" step is already done for `Age` (as a larger age has a larger numerical value), it has to be done manually for `experience`. This process will be included in the data wrangling portion in the Methods section.

### Methods

To answer our question of whether player age and experience can predict playing time, our analysis will involve data wrangling and cleaning, predictive modelling through KNN regression, and visualization of the obtained results.

First, the `players` dataset has to be tidied:
* `experience` should be of factor type rather than character, due to the ordered nature of experience: "Beginner" < "Amateur" < "Regular" < "Pro" < "Veteran"
* `Age` is a discrete numerical variable, so it should be of integer type rather than double.         
                
               
Below, the data is wrangled into a tidy format:

In [ ]:
##wrangling the relevant datasets ("players.csv") into a tidy format:

tidy_players <- players |>
    mutate(experience = factor(experience, levels = c("Beginner", "Amateur", "Regular", "Pro", "Veteran"))) |>
    mutate(Age = as.integer(Age))
head(tidy_players)

In order to narrow our data down to what we need for our research question, we need to ask what columns we would select for.            
In this case, since we are predicting playing duration from player age and experience, we need three variables: player age (`Age`), player experience (`experience`), and play duration (`played_hours`).                     
Below, we create a more targeted object containing only those three variables:

In [ ]:
target_players <- tidy_players |>
    select(experience, Age, played_hours) 
head(target_players)

##### Summarizing the data set for exploratory analysis

Below is a summary of `Age` and `played_hours`, the two numerical variables involved in our data analysis:

In [ ]:
mean_players <- target_players |>
    select(Age, played_hours) |>
    map_dfr(mean, na.rm = TRUE) |>
    as.data.frame()
colnames(mean_players) <- c("Mean Age of Participants", "Mean Duration Played (hours)")
mean_players

min_players <- target_players |>
    select(Age, played_hours) |>
    map_dfr(min, na.rm = TRUE) |>
    as.data.frame()
colnames(min_players) <- c("Minimum Age of Participants", "Minimum Duration Played (hours)")
min_players

max_players <- target_players |>
    select(Age, played_hours) |>
    map_dfr(max, na.rm = TRUE) |>
    as.data.frame()
colnames(max_players) <- c("Maximum Age of Participants", "Maximum Duration Played (hours)")
max_players

The following table summarizes the above statistics:

| Variable | Mean | Min | Max |
|----------|------|-----|-----|
| Age | 20.5206 | 8 | 50 |
| played_hours | 5.8459 | 0 |223.1 |

To understand the distribution of player experience, we count the number of players within each experience level:

In [ ]:
count <- target_players |>
    group_by(experience) |>
    summarize(n = n())
count

There are 35 beginners, 63 amateur, 36 regular, 14 pro, and 48 veteran players in the dataset.

##### Visualising the data set for exploratory analysis
The above data can also be visualized through graphs to provide a more direct understanding: 

In [ ]:
##distribution of player experience -need to add figure legend and number!!
exp_distribution <- target_players |>
    ggplot(aes(x = experience)) +
        geom_bar(stat = "count") +
        labs(x = "Player Experience", y = "Number of Players", title = "Distribution of Player Experience") +
        theme(text = element_text(size = 16))
exp_distribution